In [ ]:
import json
import time
import pandas as pd
from datetime import datetime
from pathlib import Path


def io_to_if(option_code):
    io = option_code.split('-')[0]
    if_code = io.replace('IO', 'IF')
    return if_code + '.CCFX'


def get_strike_price_from_code(option_code):
    strike_str = option_code.split('-')[2].split('.')[0]
    return int(strike_str)


def colname_add_prefix(prefix):
    def change_name(col_name):
        if col_name == 'time':
            return col_name
        return prefix + '_' + col_name
    return change_name


def get_ticks_by_trade_date(trade_date, call, put, future):
    end_dt = trade_date.replace(hour=15, minute=30)
    s_p = get_strike_price_from_code(call)
    
    fields=['time', 'current', 'volume', 'a1_p', 'a1_v', 'b1_p', 'b1_v']
    call_df = get_ticks(call, start_dt=trade_date, end_dt=end_dt, fields=fields, skip=False, df=True)
    call_df.rename(columns=colname_add_prefix('call'), inplace=True)
    put_df = get_ticks(put, start_dt=trade_date, end_dt=end_dt, fields=fields, skip=False, df=True)
    put_df.rename(columns=colname_add_prefix('put'), inplace=True)
    future_df = get_ticks(future, start_dt=trade_date, end_dt=end_dt, fields=fields, skip=False, df=True)
    future_df.rename(columns=colname_add_prefix('future'), inplace=True)
    
#     future_df.to_csv('futur_test.csv')
#     call_df.to_csv('call_test.csv')
#     put_df.to_csv('put_test.csv')
    
    df_list = [call_df, put_df, future_df]
    for df in df_list:
#         print(df[df.time.duplicated()])
        df.drop_duplicates(subset='time', inplace=True)
        df.set_index('time', inplace=True)
    new_df = pd.concat(df_list, axis=1)
    new_df['strike'] = s_p
    return new_df


def float_time_to_dt(float_time):
    main_t, ms = str(float_time).split('.')
    year = int(main_t[0: 4])
    month = int(main_t[4: 6])
    day = int(main_t[6: 8])
    hour = int(main_t[8: 10])
    minute = int(main_t[10: 12])
    second = int(main_t[12: 14])
    microsecond = int(ms) * 100000
    return datetime(year, month, day, hour, minute, second, microsecond)


def get_atm_data(file_name):
    with open(file_name, 'r') as f:
        io_atms = json.load(f)
    
    atm_df = pd.DataFrame(io_atms)
    atm_df['date'] = atm_df['date'].map(lambda date: datetime.strptime(date, '%Y%m%d'))

    for col_name in ['m_atm_call', 'm_atm_put']:
        atm_df[col_name] = atm_df[col_name].map(lambda code: code.replace("CFFEX", "CCFX"))

    atm_df['future_code'] = atm_df['m_atm_call'].map(io_to_if)
    return atm_df


def get_all_ticks_by_future(future_df):
    future_name = future_df.iloc[0]['future_code']
    ticks_df_list = []
    for _idx, s in future_df.iterrows():
        call = s['m_atm_call']
        put = s['m_atm_put']
        future = s['future_code']
        date = s['date']
        tick_df = get_ticks_by_trade_date(date, call, put, future)
        ticks_df_list.append(tick_df)
        print(f'{date}-数据下载完成,休息2秒')
        time.sleep(2)
    new_df = pd.concat(ticks_df_list)
    new_df.to_csv(f'ticks.{future_name}.csv')
    print(f'{future_name}全部下载完成，保存成功')
    return new_df


def get_all_ticks(atm_df):
    for name, group in atm_df.groupby('future_code'):
        print(name)
        print(group)
    pass

In [ ]:
filename = 'io_atm_contracts.json'
atm_df = get_atm_data(filename)
atm_df
atm_group_dict = dict(list(atm_df.groupby('future_code')))
print(atm_group_dict.keys())

In [ ]:
# f_df = atm_group_dict['IF2010.CCFX']
# f_df = atm_group_dict['IF2009.CCFX']
# f_df = atm_group_dict['IF2008.CCFX']
# f_df = atm_group_dict['IF2007.CCFX']
# f_df = atm_group_dict['IF2006.CCFX']
# f_df = atm_group_dict['IF2005.CCFX']
# f_df = atm_group_dict['IF2004.CCFX']
f_df = atm_group_dict['IF2003.CCFX']

ticks_df = get_all_ticks_by_future(f_df)

In [ ]:
# 测试
trade_date = datetime(2020, 5, 6)
df = get_ticks_by_trade_date(trade_date, 'IO2005-C-3850.CCFX', 'IO2005-P-3850.CCFX', 'IF2005.CCFX')